# Geotechnical data API - Demo

This notebook demonstrates how the soildata app of ```owimetadatabase``` can be used to retrieve geotechnical data through the API.

## Library imports

We need to import a few essential libraries first:

   - ```pandas``` for manipulation of tabular data
   - ```requests``` to send and receive HTTP requests
   - ```json``` to handle the JSON data returned by the API
   - ```os``` to retrieve environment variables
   - ```plotly``` for plotting data

In [ ]:
import pandas as pd
import requests
import json
import os
import plotly.express as px

For geotechnical data manipulation, the ```groundhog``` library is used. The modules for soil profiles and PCPT testing are loaded.

In [ ]:
from groundhog.general import soilprofile
from groundhog.siteinvestigation.insitutests.pcpt_processing import PCPTProcessing

## API access setup

### Authentication

The API is only accessible for authenticated users. To get a user account, send an email to bruno.stuyts@vub.be with your name, affiliation and use case.

Users will receive an API token which needs to be stored as the environment variable ```OWIMETA_TOKEN```. We can check that the environment variable is not empty. In case of problems, the try refreshing the environment variables before running Jupyter. Alternatively, you can just assign the value of your token to ```TOKEN``` (not recommended for security reasons).

In [ ]:
TOKEN = os.getenv('OWIMETA_TOKEN')
TOKEN

We can set up the header of the API requests as follows:

In [ ]:
head = {'Authorization': 'Token %s' % (TOKEN)}

With this header, we can authenticate all requests.

### URL prefixes

The API base URLs for the ```locations``` and ```soildata``` applications can be assigned to the ```LOCATION_URL_PREFIX``` and ```SOIL_URL_PREFIX``` variables. This avoids having to type the full URL each time we make a request.

In [ ]:
LOCATION_URL_PREFIX = "https://qmi9esf0b3.execute-api.eu-central-1.amazonaws.com/staging/api/v1/locations"
SOIL_URL_PREFIX = "https://qmi9esf0b3.execute-api.eu-central-1.amazonaws.com/staging/api/v1/soildata"

## Survey campaigns

Retrieving which survey campaigns happened on a project is done through the ```/soildata/surveycampaign/``` endpoint. The ```projectsite``` URL parameter allows filtering based on project site. Here, we can retrieve the geotechnical surveys performed at the Borssele I site.

In [ ]:
resp = requests.get('%s/surveycampaign/' % SOIL_URL_PREFIX, headers=head, params=dict(projectsite='Borssele I'))
resp

The response also contains text in the body. This text can be retrieved using the ```text``` attribute. This contains JSON with the records returned from the API. A list of records is returned in all cases. To allow faster manipulation, we can load the data into a Pandas dataframe.

In [ ]:
campaigns_df = pd.DataFrame(json.loads(resp.text))
campaigns_df

We can also retrieve a single survey campaign using the URL parameter ```campaign```. The API call is then performed as follows (example for retrieving data of the borehole investigation).

In [ ]:
resp = requests.get('%s/surveycampaign/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite='Borssele I', campaign="Borehole investigation"))
campaign_df = pd.DataFrame(json.loads(resp.text))
campaign_df

## Borehole locations

Determining where the boreholes are located is an essential step in determining the geotechnical data coverage. This data can be retrieved from the ```/soildata/testlocation/``` endpoint. Filtering per project site and survey campaign is possible.

In [ ]:
resp = requests.get('%s/testlocation/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite='Borssele I', campaign="Borehole investigation"))
testlocations_df = pd.DataFrame(json.loads(resp.text))
testlocations_df

The geographical position of these borehole locations can be visualised using Plotly.

In [ ]:
fig = px.scatter_mapbox(testlocations_df, lat='northing', lon='easting', hover_name='title',
    hover_data=['title'], zoom=10, height=500)
fig.update_layout(mapbox_style='open-street-map')
fig.show()

An API endpoint (```/soildata/testlocationproximity/```) for retrieving test locations in the vicinity of a central point is also available. We can retrieve the test locations in a radius of 500m around BH-WFS1-2A.

In [ ]:
resp = requests.get('%s/testlocationproximity/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(latitude=51.74374, longitude=3.040028, offset=0.5))
proximitylocations_df = pd.DataFrame(json.loads(resp.text))
proximitylocations_df

We can see that a geotechnical test was also performed in the vicinity of the tested location during the seafloor CPT investigation.

Furthermore, test locations in the vicinity of a profile line can be retrieved (```/soildata/testlocationprofile``` endpoint). We need to specify latitude and longitude of the start and end point and the width of the search band (in meters). We can create a profile from location BH-WFS1-2A to location BH-WFS1-6 (NW-SE profile) with a 500m search band on either side of the profile line.

In [ ]:
resp = requests.get('%s/testlocationprofile/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(lat1=51.74374, lon1=3.040028, lat2=51.70409, lon2=3.122349, offset=500))
profilelocations_df = pd.DataFrame(json.loads(resp.text))

We can also plot these locations. Their position along the profile is obvious.

In [ ]:
fig = px.scatter_mapbox(profilelocations_df, lat='northing', lon='easting', hover_name='title',
    hover_data=['title'], zoom=10, height=500)
fig.update_layout(mapbox_style='open-street-map')
fig.show()

## In-situ test data

In-situ testing returns valuable data on the geotechnical conditions at a site and in-situ data is stored in ```owimetadatabase``` in unstructured JSON fields to allow rapid retrieval of relevant data. The data has been uploaded using a standard format for common column names (e.g. ```'z [m]'``` for depth below mudline, ```'qc [MPa]'``` for cone tip resistance, ...). This allows rapid processing of the data once retrieved from the database.

### In-situ test types

We first need to know which in-situ test types exist in the database. A call to the ```/soildata/insitutesttype/``` endpoint exposes this information.

In [ ]:
resp = requests.get('%s/insitutesttype/' % SOIL_URL_PREFIX, headers=head)
insitutesttypes_df = pd.DataFrame(json.loads(resp.text))
insitutesttypes_df

### In-situ test summary data

Retrieving full data can make the HTTP requests time out if data is requested for a large number of in-situ tests. To still allow metadata on the in-situ tests to be retrieved, the endpoint ```/soildata/insitutestsummary/``` is available. This only retrieves the metadata and not the detailed test results. A listing of all seabed can be retrieved for example. URL parameters can be used for filtering.

In [ ]:
resp = requests.get('%s/insitutestsummary/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite='Borssele I', testtype="Seabed PCPT"))
insitutestsummary_df = pd.DataFrame(json.loads(resp.text))
insitutestsummary_df.head()

### In-situ test detailed data

To retrieve the detailed test data, an API call to the ```/soildata/insitutestdetail/``` can be made. To prevent timeouts, a separate call can be made for each location. For example, retrieving the downhole CPT data for location BH-WFS1-2A happens as follows:

In [ ]:
resp = requests.get('%s/insitutestdetail/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite='Borssele I', location="BH-WFS1-2A", testtype="Downhole PCPT"))
bhwfs12a_insitutest_df = pd.DataFrame(json.loads(resp.text))
bhwfs12a_insitutest_df

The CPT data itself is contained in the ```rawdata``` attribute. This is Pandas-compatible JSON, so we can load this into a dataframe:

In [ ]:
bhwfs12a_insitutest_rawdata = pd.DataFrame(bhwfs12a_insitutest_df['rawdata'].iloc[0])
bhwfs12a_insitutest_rawdata.head()

This CPT data can be loaded into a ```groundhog``` ```PCPTProcessing``` object for further processing:

In [ ]:
bhwfs12a_insitutest_cpt = PCPTProcessing(title="BH-WFS1-2A")
bhwfs12a_insitutest_cpt.load_pandas(bhwfs12a_insitutest_rawdata, push_key="Push")

The CPT data can be plotted:

In [ ]:
bhwfs12a_insitutest_cpt.plot_raw_pcpt(u2_range=(-0.5, 2.5), u2_tick=0.5)

### Batch lab test data

Batch lab test data is laboratory test data carried out in bulk, often on-board the site investigation vessel. The available test types can be retrieved with a call to the ```/soildata/batchlabtesttype/``` endpoint.

In [ ]:
resp = requests.get('%s/batchlabtesttype/' % SOIL_URL_PREFIX, headers=head)
batchlabtesttypes_df = pd.DataFrame(json.loads(resp.text))
batchlabtesttypes_df

We can retrieve either summary (```/soildata/batchlabtestsummary/``` endpoint) or detailed (```/soildata/batchlabtestdetail/```) data. As an example, we can retrieve all water contents for location BH-WFS1-2A at the Borssele I offshore wind farm.

In [ ]:
resp = requests.get('%s/batchlabtestdetail/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite='Borssele I', location="BH-WFS1-2A", testtype="Water content"))
bhwfs12a_batchlabtest_df = pd.DataFrame(json.loads(resp.text))
bhwfs12a_batchlabtest_df

The ```rawdata``` attribute contains the measurements in JSON format.

In [ ]:
bhwfs12a_batchlabtest_data = pd.DataFrame(bhwfs12a_batchlabtest_df['rawdata'].iloc[0])
bhwfs12a_batchlabtest_data.head()

## Sample test data

Data from advanced laboratory tests is stored in the database in the ```sampletest``` table. The API can also be used to access this data.

First, we can retrieve a listing of the samples on a project using a call to the ```/soildata/geotechnicalsample/``` endpoint.

In [ ]:
resp = requests.get('%s/geotechnicalsample/' % SOIL_URL_PREFIX, headers=head, params=dict(projectsite="Borssele I"))
geotechnicalsample_df = pd.DataFrame(json.loads(resp.text))
geotechnicalsample_df.head()

The sample test types can be retrieved using a call to the ```/soildata/sampletesttype/``` endpoint:

In [ ]:
resp = requests.get('%s/sampletesttype/' % SOIL_URL_PREFIX, headers=head)
sampletesttypes_df = pd.DataFrame(json.loads(resp.text))
sampletesttypes_df

As an example, we can retrieve the bender element test results on sample W18 using a call to the ```/soildata/sampletestdetail/``` endpoint:

In [ ]:
resp = requests.get('%s/sampletestdetail/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite="Borssele I", sample="W18", testtype="Bender element"))
benderelement_W18_df = pd.DataFrame(json.loads(resp.text))
benderelement_W18_df.head()

The test data is stored in the ```rawdata``` attribute.

In [ ]:
benderelement_W18_df['rawdata'].iloc[0]

The measured value of small-strain shear modulus $ G_{max} $ can be retrieved as follows. This is the value after the isotropic consolidation stage.

In [ ]:
benderelement_W18_df['rawdata'].iloc[0]['Gmax selected [MPa]']

## Soil profiles

Soil profile retrieval is relatively straightforward using the API. To prevent timeout API requests, a ```/soildata/soilprofilesummary/``` and ```/soildata/soilprofiledetail/``` are provided for metadata-only and full data retrieval respectively.

First, we can retrieve the metadata for all soil profiles at the Borssele I site:

In [ ]:
resp = requests.get('%s/soilprofilesummary/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite="Borssele I"))
soilprofile_summary_df = pd.DataFrame(json.loads(resp.text))
soilprofile_summary_df

We can create a soil profile for the BH-WFS1-2A location:

In [ ]:
resp = requests.get('%s/soilprofiledetail/' % SOIL_URL_PREFIX, headers=head,
                    params=dict(projectsite="Borssele I", location="BH-WFS1-2A"))
soilprofile_detail_df = pd.DataFrame(json.loads(resp.text))
soilprofile_detail_df

The layering is defined in the ```soillayer_set``` attribute. We can convert this into a Pandas dataframe.

In [ ]:
bhwfs12a_boreholelog = pd.DataFrame(soilprofile_detail_df['soillayer_set'].iloc[0]).sort_values('start_depth')
bhwfs12a_boreholelog

We can import this soil profile into a ```groundhog``` ```SoilProfile``` object. In order to do this, we first need to rename the ```start_depth``` and ```end_depth``` columns to ```Depth from [m]``` and ```Depth to [m]```. The column ```soiltype_name``` can be renamed to ```Soil type```.

In [ ]:
bhwfs12a_boreholelog.rename(
    columns={'start_depth': 'Depth from [m]', 'end_depth': 'Depth to [m]', 'soiltype_name': 'Soil type'},
    inplace=True)

We can then create a ```SoilProfile``` object from the dataframe:

In [ ]:
bhwfs12a_soilprofile = soilprofile.profile_from_dataframe(bhwfs12a_boreholelog)
bhwfs12a_soilprofile

We can plot a mini-log of this profile. We need to define a mapping for the soil type:

In [ ]:
soiltypecolors = {
    "SAND": 'yellow',
    "CLAY": 'brown',
    "Clayey SAND": 'orange',
    "Silty SAND": '#fcba03'
}

bhwfs12a_minilog = bhwfs12a_soilprofile.plot_profile(parameters=((),), fillcolordict=soiltypecolors)